In [2]:
pip install langchain langchain-core langchain-community langchain-google-genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

c:\Users\Kunj Patel\Desktop\Langchian Models\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [9]:
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke("Today's top news of india")
results

c:\Users\Kunj Patel\Desktop\Langchian Models\venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


'1 day ago · The Supreme Court is hearing cases challenging the electoral rolls revision in Bihar; PM Modi is likely to meet US President Donald Trump in September. Top News Headlines of August … Aug 3, 2025 · Get the latest breaking news from India covering politics, economy, social issues, technology, sports, and more. Stay updated with in-depth analysis and top headlines from across … 4 days ago · Stay informed with breaking news and latest news on politics, business, entertainment sports, science, technology along with news updates from around the world. Jul 7, 2025 · Breaking News Live: Stay connected with this page for the latest and fastest updates on the biggest news stories of the day. Get real-time alerts as major events unfold. 3 days ago · Stay informed with the latest national news from India. Get today’s top headlines, breaking news, key developments, business updates, and trending national stories—live and in …'

In [18]:
from langchain_core.tools import tool
import requests
@tool
def get_weather(city:str):
  """
  This functions fetches the current weather condition of given city
  """ 
  url = f"https://goweather.herokuapp.com/weather/{city}"
  
  response = requests.get(url)
  
  return response.json()

In [20]:
result = get_weather.run("Surat")  # .run() is needed because @tool wraps it
print(result)

{'temperature': '+30 °C', 'wind': '19 km/h', 'description': 'Patchy rain nearby', 'forecast': [{'day': '1', 'temperature': '32 °C', 'wind': '18 km/h'}, {'day': '2', 'temperature': '+26 °C', 'wind': '10 km/h'}, {'day': '3', 'temperature': ' °C', 'wind': '7 km/h'}]}


In [21]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm.invoke("Hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--c131dec0-124c-452c-ac26-0a4198692065-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [22]:
from langchain.agents import create_react_agent,  AgentExecutor
from langchain import hub

In [23]:
#Step2 : Pul the ReAct prompt from langchain Hub

prompt = hub.pull("hwchase17/react") 

c:\Users\Kunj Patel\Desktop\Langchian Models\venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [24]:
prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [25]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool,get_weather],
    prompt=prompt
)

In [26]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
  agent = agent,
  tools = [search_tool,get_weather],
  verbose = True
)

In [27]:
#Step 5: Invoke
response = agent_executor.invoke({"input": "Where is Surat and find the current temperature of the city "})



> Entering new AgentExecutor chain...
Thought: I need to find the location of Surat and then get its current temperature.  I'll first use a search engine to confirm the location and then use the weather API.

Action: duckduckgo_search
Action Input: "Where is Surat?"

c:\Users\Kunj Patel\Desktop\Langchian Models\venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Surat (Gujarati: [suɾət]) is a city in the western Indian state of Gujarat. The word Surat directly translates to face in Gujarati, Hindi and Urdu. Located on the banks of the river Tapti near its confluence with the Arabian Sea, it used to be a large seaport. Aug 5, 2025 · Surat is located on the southern bank of Tapi River in Gujarat. It is well-connected to all parts of India. Surat has attracted merchants and traders due to its seaports from ancient times. A... Surat is a city located on the western part of India in the state of Gujarat.. Surat lies between 21°10′12.864° North latitudes and 72°49′51.819° East longitudes. The city covers an area of about 474.185 km2 with a population of 6936534 people. Surat sits on the Tapti River’s banks in Gujarat’s west, near where it meets the Arabian Sea. It is found at the coordinates of 21°12′18″N and 72°50′24″E. Surat is one of the largest cities in the northwestern part of India, which is situated in the state of Gujarat. The city was buil

In [28]:
response["output"]

'Surat is a city in the western Indian state of Gujarat. The current temperature in Surat is +30°C, with patchy rain nearby and a wind speed of 19 km/h.'